In [2]:
from openai import OpenAI

client = OpenAI(
  api_key=""
)

assistant = client.beta.assistants.create(
  name="SBIR Assistant",
  instructions="You are an expert on government grants and funding. You are able to answer questions about the grants and funding available to small businesses.",
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}],
)

In [ ]:
# Create a vector store caled "SBIR Grants"
vector_store = client.beta.vector_stores.create(name="SBIR Grants")

# Ready the files for upload to OpenAI
file_paths = ["./DoD_grants.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [4]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [9]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Use the PDF file Dod_grants that is in your vector store. Only use that to answer this prompt: Give me all grants related to supply chain vulnerabilities."
)

# thread = client.beta.threads.create(
#   messages=[
#     {
#       "role": "user",
#       "content": "Use the PDF file Dod_grants that is in your vector store. Only use that to answer this prompt: Give me all grants related to supply chain vulnerabilities.",
#     }
#   ]
# )

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

None


In [10]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Here are the grants related to supply chain vulnerabilities from the provided document:

1. **Resilience against Supply Chain Cyber Vulnerabilities**
   - **Objective**: Develop a technology that ensures computing hardware technologies integrated into future combat systems are trustworthy and cyber secure.
   - **Description**: This grant focuses on enhancing the resilience of shipboard computing infrastructure and ensuring the security, robustness, trustworthiness, and performance against potential cyber threats in the supply chain[0].

2. **Smart Contracts for Supply Chain Risk Management (SCRM)**
   - **Objective**: Develop an acquisition and sustainment contracting framework to implement smart contracts for SCRM using blockchain technology across Strategic Systems Programs (SSP).
   - **Description**: This project aims to improve visibility of sub-tier vendors in the supply chain to enhance resilience and respond to potential risks. The initiative aligns with actions taken in respo

In [11]:
# testing follow up question
follow_up = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Tell me more about the first grant"
)

follow_up_run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=follow_up_run.id))

message_content = messages[0].content[0].text
print(message_content.value)

Here are the details regarding the grant titled **Resilience against Supply Chain Cyber Vulnerabilities**:

- **Objective**: The primary aim of this grant is to develop technology that ensures that computing hardware technologies integrated into future combat systems are trustworthy and cyber secure. This is critical for maintaining the integrity and functionality of defense systems within potentially hostile environments.

- **Description**: The shipboard computing infrastructure currently comprises over 3,000 Central Processing Unit (CPU) cores distributed across various military-grade cabinets situated in multiple spaces within a ship. This distributed setup is designed to enhance operational survivability in the event of damage to specific components. However, this inherent complexity brings significant challenges in maintaining security, robustness, trustworthiness, and performance of the computing infrastructure. 

The proposed technology focuses on infrastructure resilience, whi